In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms

# CCN

In [2]:
class CNN(nn.Module):
    def __init__(self, in_channels=1, num_classes=10):
        super(CNN, self).__init__()
        
        # same conv : 입출력 크기 유지
        self.conv1 = nn.Conv2d(
            in_channels=1, out_channels=8,
            kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)
        )
        self.pool = nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2))
        self.conv2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        self.fc1 = nn.Linear(16*7*7, num_classes)
    
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fc1(x)
        return x

# Save & Load model

In [3]:
def save_checkpoint(state, filename='my_checkpoint.pth.tar'):
    """save checkpoint"""
    print('=> Saving checkpoint')
    torch.save(state, filename)

In [4]:
def load_checkpoint(checkpoint):
    print('=> Loading checkpoint')
    model.load_state_dict(checkpoint['state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer'])

In [5]:
model = CNN(784, 10)
x = torch.randn(64, 1, 28, 28)
model(x).shape

torch.Size([64, 10])

# Settings

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

in_channels = 1
num_classes = 10
learning_rate = 0.001
batch_size = 64
num_epochs = 1
load_model = True

# Load Data

In [7]:
train_dataset = datasets.MNIST(
    root='dataset/',
    train=True, 
    transform=transforms.ToTensor(), 
    download=True
)

test_dataset = datasets.MNIST(
    root='dataset/',
    train=False, 
    transform=transforms.ToTensor(), 
    download=True
)

In [8]:
train_loader = DataLoader(
    dataset=train_dataset,
    batch_size=batch_size,
    shuffle=True
)

test_loader = DataLoader(
    dataset=test_dataset,
    batch_size=batch_size,
    shuffle=True
)

# Initalize

In [9]:
model = CNN(
    in_channels=in_channels,
    num_classes=num_classes,
).to(device)
model

CNN(
  (conv1): Conv2d(1, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=784, out_features=10, bias=True)
)

In [10]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

if load_model:
    load_checkpoint(torch.load('my_checkpoint.pth.tar'))

=> Loading checkpoint


# Train

In [11]:
for epoch in range(num_epochs):
    
    # save models
    checkpoint = {
        'state_dict': model.state_dict(),
        'optimizer': optimizer.state_dict()
    }
    save_checkpoint(checkpoint)
    
    for batch_idx, (data, targets) in enumerate(train_loader):
        
        # cuda
        data = data.to(device)
        targets = targets.to(device)
        
        # forward
        scores = model(data)
        loss = criterion(scores, targets)
        
        # backward
        optimizer.zero_grad()
        loss.backward()
        
        # gradient descent
        optimizer.step()
    
    print(f'epoch: {epoch} / {num_epochs}')

=> Saving checkpoint
epoch: 0 / 1


In [12]:
def check_acc(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()
    
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device)
            y = y.to(device)
            
            scores = model(x)
            _, pred = scores.max(1)
            num_correct += (pred == y).sum()
            num_samples += pred.size(0)
        
        print(f'{num_correct} / {num_samples}, acc: {float(num_correct) / float(num_samples) * 100:.2f}')

In [13]:
check_acc(train_loader, model)

57742 / 60000, acc: 96.24


In [14]:
check_acc(test_loader, model)

9636 / 10000, acc: 96.36
